In [1]:
##基于用户兴趣标签的推荐
import math
import numpy as np 
tag_size = 10 
hit_count = 0

In [15]:
##导入uid
def load_uid():
    uid_list = []
    with open('../../data/uid.txt','r',encoding = 'utf-8') as f1:
        while True:
            line = f1.readline()
            if not line:
                break
            uid_list.append(line.strip())
        f1.close()
    return uid_list

In [16]:
##读入训练集
def load_train():
    uid_wid = {}
    original = {}
    filename = '../../data/train/'+str(uid)+'.dat'
    with open(filename,'r',encoding='utf-8') as f:
        while True:
            line = f.readline()
            if not line:
                break
            line = eval(line)
            wid = line['id']
            userid = line['userid']
            originalid = line['original']
            if 'id' not in originalid:
                original[wid] = wid
            else:
                original[wid] = originalid['id']
            if userid not in uid_wid:
                uid_wid[userid] = []
            uid_wid[userid].append(wid)
        f.close()
    return uid_wid,original  

In [17]:
##读入微博文本
def load_text():
    text = {}
    filename = '../../data/text/before_split/'+str(uid)+'.txt'
    with open(filename,'r',encoding='utf-8') as f:
        line = f.readline()
        text = eval(line)
        f.close()
    for item in text.keys():
        text[item] = text[item].replace(' ','') ##去掉文本中的空格
    return text  

In [18]:
#将原始微博与转发微博合并
def combine_text(item):
    text = item['text']
    original = item['original']
    if original.get('text'):
        text = text + "//" + original['text']
    return text

In [19]:
##读入测试集
def load_test():
    test = [] ##测试集
    user_test = [] ##目标用户微博
    filename = '../../data/test/'+str(uid)+'.dat'
    with open(filename,'r',encoding='utf-8') as f:
        while True:
            line = f.readline()
            if not line:
                break
            line = eval(line)
            wid = line['id']
            userid = line['userid']
            originalid = line['original']
            if 'id' not in originalid:
                original[wid] = wid
            else:
                original[wid] = originalid['id']
            if userid not in uid_wid:
                uid_wid[userid] = []
            uid_wid[userid].append(wid)
            test.append(wid)
            if userid == uid:
                user_test.append(wid)
            t = combine_text(line)
            text[wid] = t
        f.close()
    return test,user_test

In [20]:
##最大最小归一化
def max_min(data):
    ma = 0
    mi = 100
    for item in data.keys():
        if data[item] > ma:
            ma = data[item]
        if data[item] < mi:
            mi = data[item]
    for item in data.keys():
        data[item] = (data[item] - mi) / (ma-mi)
    return data

In [21]:
##保存召回结果
def save_result(result):
    filename = './weibo_by_tag/'+str(uid)+'.dat'
    with open(filename,'w',encoding='utf-8') as f:
        for item in result:
            f.write(str(item)+'\n')
        f.close()

In [22]:
def get_result(result):
    weibo = []##推荐集中用户产生行为的微博
    count = 0
    for wid in user_test:
        weibo.append(original[wid])
    for item in result:
        if item in weibo:
            count += 1
            hit_set.add(item)
    save_result(result)
    return count

In [23]:
def load_tfidf():
    tfidf = {}
    filename = './user_interest_tag/'+ str(uid) + '_usertag_by_tfidf.dat'
    with open(filename,'r',encoding = 'utf-8') as f:
        count = 0
        while True:
            count += 1
            line = f.readline().strip()
            if not line:
                break
            line = line.split('\t')
            tfidf[line[0]] = float(line[1])
            if count >= tag_size:
                break
        f.close()
    return tfidf

In [24]:
##导入对照组标签
def load_textrank():
    textrank = {}
    filename = './user_interest_tag/'+ str(uid) + '_usertag_by_textrank.dat'
    with open(filename,'r',encoding = 'utf-8') as f:
        count = 0
        while True:
            count += 1
            line = f.readline().strip()
            if not line:
                break
            line = line.split('\t')
            textrank[line[0]] = float(line[1])
            if count >= tag_size:
                break
        f.close()
    return textrank

In [25]:
##导入实验组标签
def load_textrank_extend():
    textrank = {}
    filename = './user_interest_tag/'+ str(uid) + '_usertag_by_textrank_extend.dat'
    with open(filename,'r',encoding = 'utf-8') as f:
        count = 0
        while True:
            count += 1
            line = f.readline().strip()
            if not line:
                break
            line = line.split('\t')
            textrank[line[0]] = float(line[1])
            if count >= tag_size:
                break
        f.close()
    return textrank

In [26]:
def textrank_recall():
   # print("textrank算法：")
    textrank = load_textrank()
    textrank = max_min(textrank)
    score = {}
    for i in test:
        score[i] = 0
        for tag in textrank.keys():
            if tag in text[i]:
                score[i] += textrank[tag]
    score = sorted(score.items(),key=lambda x:x[1],reverse=True)
    result = set()
    n = len(score)
    for i in range(0,n):
        if score[i][1] <= 0:
            break
        result.add(original[score[i][0]]) 
        if len(result) >= num:
            break
    hit_count = get_result(result)
    return result,hit_count

In [27]:
def textrank_extend_recall():
    #print("textrank扩展算法：")
    textrank = load_textrank_extend()
    textrank = max_min(textrank)
    #print(textrank)
    score = {}
    for i in test:
        score[i] = 0
        for tag in textrank.keys():
            l = tag.split(',')
            flag = 0
            for tt in l:
                if tt not in text[i]:
                    flag = 1
                    break
            if flag == 0:
                score[i] += textrank[tag]
    score = sorted(score.items(),key=lambda x:x[1],reverse=True)
    result = set()
    n = len(score)
    for i in range(0,n):
        if score[i][1] <= 0:
            break
        result.add(original[score[i][0]])  ##推荐集里为原创微博(可能推荐的多条微博实际为一条原创微博的转发)
        if len(result) >= num:
            break
    #print(len(result))
    hit_count = get_result(result)
    return result,hit_count

In [28]:
if __name__ == '__main__':
    uid_list = load_uid()
    for num in [10,20,30,40,50]:
        hit_count1 = 0
        hit_count2 = 0
        hit_set = set()
        for uid in uid_list:
            #print(uid)
            uid = int(uid)
            uid_wid,original = load_train()  ##读取训练集
            text = load_text() ##读取微博文本内容
            test,user_test = load_test()  ##读取测试集
            result,count1 = textrank_recall()  ##对照组召回
            extend_result,count2 = textrank_extend_recall() ##实验组召回
            hit_count1 += count1
            hit_count2 += count2
            save_result(extend_result)
        print("TTI top{},recall: {}".format(num,hit_count1/800)) 
        print("TTI top{},precision: {}".format(num,hit_count1/(num*20)))
        print("TE top{},recall: {}".format(num,hit_count2/800)) 
        print("TE top{},precision: {}".format(num,hit_count2/(num*20)))

TTI top10,recall: 0.0475
TTI top10,precision: 0.19
TE top10,recall: 0.055
TE top10,precision: 0.22
TTI top20,recall: 0.08
TTI top20,precision: 0.16
TE top20,recall: 0.0975
TE top20,precision: 0.195
TTI top30,recall: 0.105
TTI top30,precision: 0.14
TE top30,recall: 0.14
TE top30,precision: 0.18666666666666668
TTI top40,recall: 0.12875
TTI top40,precision: 0.12875
TE top40,recall: 0.16
TE top40,precision: 0.16
TTI top50,recall: 0.155
TTI top50,precision: 0.124
TE top50,recall: 0.16875
TE top50,precision: 0.135
